<a href="https://www.kaggle.com/code/howecnchen/keras-lora-gemma?scriptVersionId=165518095" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Try [Fine-tune Gemma models in Keras using LoRA](https://ai.google.dev/gemma/docs/lora_tuning) without CoLab.

在非 CoLab 环境尝试 [Fine-tune Gemma models in Keras using LoRA](https://ai.google.dev/gemma/docs/lora_tuning) 。

**All code descriptions are generated by Gemini. I can't be sure if this is correct**.

**下文中对于代码的描述都是基于Gemini生成的。我暂时不能保证其正确性**。

Download ant Install the `Keras-NLP`.

下载安装`Keras-NLP`。

In [ ]:
!pip install -U keras-nlp
!pip install -U keras

In [ ]:
import keras
import keras_nlp
import numpy as np

使用 `Keras-NLP` 库来加载预训练的 `Gemma CausalLM` 模型并进行文本生成。

首先，它导入必要的库并加载模型。

然后，它打印模型的摘要，并使用模型生成文本，遵循特定的提示和最大长度限制。

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

gemma_lm.summary()
gemma_lm.generate("Keras is a", max_length=30)

`keras_nlp.models.GemmaCausalLM`从 `keras_nlp.models` 模块导入 `GemmaCausalLM` 类。
该类允许用户使用 `Gemma`，一个预训练的因果语言模型，专门设计用于文本生成任务，并能够控制事实一致性。
具体的内容可以参考[gemma_causal_lm.py](https://github.com/keras-team/keras-nlp/blob/v0.8.2/keras_nlp/models/gemma/gemma_causal_lm.py#L30)

`gemma_lm = GemmaCausalLM.from_preset("gemma_2b_en")`通过加载 "gemma_2b_en" 预设来创建一个 `gemma_lm` 对象。
这指的是一个特定的预训练 `Gemma` 模型版本，可能是一个 20 亿参数的英文模型。

`gemma_lm.summary()`将打印模型架构的摘要，包括层数、参数数量、输入/输出形状等详细信息。但是，具体输出取决于使用的 `Keras-NLP` 版本。

`gemma_lm.generate("Keras is a", max_length=30)`调用 `gemma_lm` 对象上的 `generate` 方法。它提供起始短语 "`Keras is a"` 并将 `max_length` 参数设置为 30，指示模型应该生成的最大单词数。然后，模型将尝试造句，同时根据其训练数据保持事实一致性。

这一行的输出将是一个包含生成文本的字符串，遵循提示 "`Keras is a`"。例如，模型可能会生成以下内容：

* Keras 是一个强大的深度学习库。
* Keras 是 TensorFlow 的一个用户友好 API。
* Keras 是一个用于构建神经网络的流行框架。

具体输出取决于模型的内部知识和生成过程中的随机元素。

基于原文，获取用于训练的[数据库](https://huggingface.co/datasets/databricks/databricks-dolly-15k)并读取到内存。

注意：原文中读取了1000组数据。但是，处理这么多数据会导致这个 `Notebook` 执行时间超过24小时。因此，我把1000减少到了10。

In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 10 training examples, to keep it fast.
# 1000 training examples cost about 24 hours. So I reduce it to 10 which about 10 minutes.
data = data[:10]
print(data)

基于之前的代码，使用加载的 `gemma_lm` 模型进行文本生成，并使用特定的提示和采样策略。

这段代码的输出将是一个包含生成响应的字符串，遵循格式 “Tope 是一种鱼类。”或 “Rope 不是一种鱼类。”。具体的响应将取决于模型的内部知识和生成过程中涉及的随机元素，这些元素会受到 `top-k` 采样策略的影响。

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

`template.format`使用模板来格式化模型的提示。
它插入指令“`Which is a species of fish? Tope or Rope`”并将响应设置为一个空字符串。

`sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)`从 `keras_nlp.samplers` 模块创建一个 `TopKSampler` 对象。
该采样器在生成过程中为每个单词选择 `k` 个（这里是 5 个） 最可能的延续，从而促进输出更加多样化和潜在的信息丰富。
`seed` 参数设置为 2，这有助于控制采样过程的随机性并确保每次运行代码时输出略有不同。

`gemma_lm.compile(sampler=sampler)`使用指定的采样器编译 `gemma_lm` 模型。这设置了文本生成过程中使用的采样策略。

`print(gemma_lm.generate(prompt, max_length=256))`调用 `gemma_lm` 对象上的 `generate` 方法。
它提供之前创建的 `prompt` 并将 `max_length` 设置为 256，指定模型可以生成的单词的最大数量。

修改 `gemma_lm` 模型，然后显示其摘要。

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

`gemma_lm.backbone.enable_lora(rank=4)`调用 `gemma_lm` 对象的 `backbone` 属性上的 `enable_lora` 方法。
* `backbone` 指的是 `Gemma` 模型中的核心 `transformer` 架构。
* 在 `backbone` 上启用 `LoRA` (`Low Rank Adaptation`) 引入了一种 参数高效的微调 技术。
* `rank` 参数设置为 `4`，指定 `LoRA` 中使用的分解矩阵的秩，该秩控制模型大小和性能之间的权衡。

`gemma_lm.summary()`输出模型架构的摘要。由于启用了 `LoRA`，它可能与之前的摘要不同。具体的变化将取决于 `Keras-NLP` 版本和库中 `LoRA` 的实现细节。

以下是一些您可能会观察到的潜在变化：

* 与原始模型相比，可训练参数的数量可能会显着减少。
* 摘要中可能包含与 `LoRA` 相关的附加层或信息。

注意： 启用 `LoRA` 适用于您希望在最小化可训练参数数量的同时将预训练模型适应特定任务的微调场景。在使用 `LoRA` 之前，务必考虑模型效率和潜在性能影响之间的权衡。

**分解矩阵** 是指将一个矩阵表示为多个矩阵的乘积。常见的分解矩阵方法包括：

* LU 分解: 将矩阵 A 分解为一个下三角矩阵 L 和一个上三角矩阵 U，即 A = LU。
* QR 分解: 将矩阵 A 分解为一个正交矩阵 Q 和一个上三角矩阵 R，即 A = QR。
* 奇异值分解 (SVD): 将矩阵 A 分解为三个矩阵 U、Σ 和 V，即 A = UΣV^T，其中 U 和 V 是正交矩阵，Σ 是一个对角矩阵。

**秩** 是指矩阵的行向量或列向量的最大线性无关组数。换句话说，秩是矩阵可以表示的独立方程式的最大数量。

**例子**:

* 对于矩阵 `A = [1 2; 3 4]`，其秩为 2。因为 A 的两个行向量是线性相关的，所以秩不能大于 2。
* 对于矩阵 `B = [1 2 3; 4 5 6; 7 8 9]`，其秩为 3。因为 B 的三个行向量是线性无关的，所以秩为 3。

**分解矩阵和秩的应用**:

* 求解线性方程组: 分解矩阵可以用于求解线性方程组，例如使用 LU 分解或 QR 分解。
* 矩阵逆: 矩阵的逆可以通过其秩和分解矩阵来计算。
* 图像处理: 奇异值分解可以用于图像压缩和去噪。
* 自然语言处理: 奇异值分解可以用于文本相似度计算和主题建模。

**总结**:

分解矩阵和秩是线性代数中的重要概念，它们在许多领域都有应用。

接下来，使用通用的训练设置来调整 `gemma_lm` 模型以适应任务。
* `AdamW` 是基于 `Transformer` 的语言模型中常用的优化器。
* 为了考虑内存，调整了序列长度。
* 使用加权指标来处理潜在不平衡的训练数据集。

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

`gemma_lm.preprocessor.sequence_length = 512`将模型可以处理的文本序列的最大长度限制为 512 个标记。
这通常是为了提高内存效率，特别是在使用像 `Gemma` 这样的大型语言模型时。

`keras.optimizers.AdamW`设置了用于训练模型的优化器 `AdamW`。
* `AdamW` 是 `Adam` 优化器的变体，包括权重衰减。
* 权重衰减是一种正则化技术，有助于防止过拟合。
* 偏差和层归一化参数 (`scale`) 被排除在权重衰减之外，因为它们通常需要不同的学习率。

`optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])`告诉 `AdamW` 优化器将名为 “`bias`” 和 “`scale`” 的参数从权重衰减中排除。这些参数通常与 批量归一化层 相关联，而批量归一化层经常用于像 `Gemma` 这样的 `Transformer` 模型。将它们从权重衰减中排除有助于：
* 保持它们的原始学习率: 权重衰减会影响可训练参数的学习率。排除它们允许根据需要进行独立的学习率调整。
* 保持它们的有效性: 将权重衰减应用于 “`bias`” 和 “`scale`” 可能会对批量归一化层的性能产生负面影响。

`gemma_lm.compile`编译模型，并指定了：
* 要使用的损失函数：`SparseCategoricalCrossentropy`，适用于每个样本都属于一个类的多类分类任务。
* 优化器：配置的 `AdamW` 优化器。
* 训练过程中要跟踪的指标：`SparseCategoricalAccuracy`，根据多类预测来衡量准确性。

`gemma_lm.fit(data, epochs=1, batch_size=1)`使用提供的 data 开始模型训练过程。
* `epochs=1` 表示只遍历整个数据集一次。
* `batch_size=1` 每次处理一个文本序列，通常用于大型模型或内存有限的情况。

训练之后，重复一次前面的文本生成过程。查看前后两次是否发生了变化。
这里因为大大消减了训练数据量，可能对结果的影响微乎其微。

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

保存 `gemma_lm` 模型的当前状态，包括其架构、权重以及可能适用的优化器配置。
保存模型允许您稍后加载它以供进一步使用或评估。

In [ ]:
#import os
#module_name = "dolly-15k.h5"
#gemma_lm.save(module_name, overwrite=True)